In [119]:
import pandas as pd
from pandas.tseries.offsets import DateOffset

sea_level = pd.read_csv("inputs/sealevel.csv")
data = pd.read_csv("inputs/Zip_homes.csv")

#Housing price data
transposed_data = data.T

transposed_data.reset_index(inplace=True)
new_header = transposed_data.iloc[0]
transposed_data = transposed_data[1:]
transposed_data.columns = new_header

transposed_data.rename(columns={'RegionName': 'Year'}, inplace=True)
transposed_data.set_index('Year', inplace=True)

In [120]:
#Sea Level Data
filtered_data = sea_level[::3]

years = filtered_data['Year'].unique()
limited_rows = []

for year in years:
    year_data = filtered_data[filtered_data['Year'] == year]
    limited_rows.append(year_data.iloc[:12])

filtered_data = pd.concat(limited_rows)

filtered_data['Year'] = pd.to_datetime(filtered_data['Year'], format='%Y') + filtered_data.groupby('Year').cumcount().apply(lambda x: DateOffset(months=x))
filtered_data['Year'] = filtered_data['Year'].dt.to_period('M').astype(str)

filtered_data = filtered_data[~filtered_data.index.duplicated(keep='first')]

filtered_data.set_index('Year', inplace=True)

C:\Users\Linh Thai\AppData\Local\Temp\ipykernel_25784\1111962629.py:13: PerformanceWarning: Adding/subtracting object-dtype array to DatetimeArray not vectorized.
  filtered_data['Year'] = pd.to_datetime(filtered_data['Year'], format='%Y') + filtered_data.groupby('Year').cumcount().apply(lambda x: DateOffset(months=x))


In [121]:
# Merge two data sets
merged_data = pd.merge(transposed_data, filtered_data, how='outer', left_index=True, right_index=True)
merged_data.rename(columns={'Year': 'Date'}, inplace=True)
merged_data.index = merged_data.index.rename('Date')

merged_data.to_csv('inputs/merged_data.csv')

print(merged_data.head())

         10025.0  60657.0  10023.0  60614.0  79936.0  10002.0  60640.0  \
Date                                                                     
1993-01      NaN      NaN      NaN      NaN      NaN      NaN      NaN   
1993-02      NaN      NaN      NaN      NaN      NaN      NaN      NaN   
1993-03      NaN      NaN      NaN      NaN      NaN      NaN      NaN   
1993-04      NaN      NaN      NaN      NaN      NaN      NaN      NaN   
1993-05      NaN      NaN      NaN      NaN      NaN      NaN      NaN   

         77084.0  94109.0  75070.0  ...  76005.0  81225.0  \
Date                                ...                     
1993-01      NaN      NaN      NaN  ...      NaN      NaN   
1993-02      NaN      NaN      NaN  ...      NaN      NaN   
1993-03      NaN      NaN      NaN  ...      NaN      NaN   
1993-04      NaN      NaN      NaN  ...      NaN      NaN   
1993-05      NaN      NaN      NaN  ...      NaN      NaN   

         TotalWeightedObservations  GMSL_noGIA  StdDe

In [123]:
filtered_merged_data = merged_data.loc['2010-01':'2017-09']

print(filtered_merged_data.head())

filtered_merged_data.to_csv('inputs/filtered_merged_data.csv')

          10025.0   60657.0  10023.0  60614.0   79936.0  10002.0  60640.0  \
Date                                                                        
2010-01  775000.0  349900.0      NaN      NaN  134700.0      NaN      NaN   
2010-02  725000.0  349000.0      NaN      NaN  130000.0      NaN      NaN   
2010-03  762500.0  339619.5      NaN      NaN  129900.0      NaN      NaN   
2010-04  725000.0  349000.0      NaN      NaN  129900.0      NaN      NaN   
2010-05  732500.0  334500.0      NaN      NaN  131450.0      NaN      NaN   

          77084.0  94109.0   75070.0  ...  76005.0  81225.0  \
Date                                  ...                     
2010-01  119900.0      NaN  191315.0  ...      NaN      NaN   
2010-02  119850.0      NaN  194900.0  ...      NaN      NaN   
2010-03  121950.0      NaN  190995.0  ...      NaN      NaN   
2010-04  120000.0      NaN  195000.0  ...      NaN      NaN   
2010-05  120000.0      NaN  194950.0  ...      NaN      NaN   

         TotalWeig

In [124]:
import pandas as pd

Coastal = [94015.0, 93950.0, 93109.0, 33137.0, 33129.0, 33131.0, 33308.0, 33062.0, 70124.0, 70122.0, 70126.0, 77505.0, 77058.0, 77015.0, 10069.0, 10010.0, 10305.0, 10314.0, 11214.0, 11209.0]
Inland = [94014.0, 93940.0, 93108.0, 33127.0, 33145.0, 33130.0, 33060.0, 70118.0, 70119.0, 77504.0, 77062.0, 10023.0, 10003.0, 10304.0, 11204.0]
Combined = [94014.0, 94015.0, 93940.0, 93950.0, 93108.0, 93109.0, 77504.0, 77505.0, 33127.0, 33137.0, 33145.0, 33129.0, 33130.0, 33131.0, 33309.0, 33308.0, 33060.0, 33062.0, 70118.0, 70124.0, 70119.0, 70122.0, 70116.0, 70126.0, 77062.0, 77058.0, 77020.0, 77015.0, 77078.0, 77044.0, 10304.0, 10305.0, 10306.0, 10314.0, 11204.0, 11214.0, 11219.0, 11209.0, 10023.0, 10069.0, 10003.0, 10010.0]
additional_columns = [
    "TotalWeightedObservations",
    "GMSL_noGIA",
    "StdDevGMSL_noGIA",
    "SmoothedGSML_noGIA",
    "GMSL_GIA",
    "StdDevGMSL_GIA",
    "SmoothedGSML_GIA",
    "SmoothedGSML_GIA_sigremoved",
]

selected_columns = Coastal + Inland + additional_columns

filtered_zip = filtered_merged_data.loc[:, selected_columns]
filtered_zip = filtered_zip.reset_index()

# Transpose the DataFrame
transposed_filtered_zip = filtered_zip.transpose()

# Reset the index to create a new column with the index values
transposed_filtered_zip = transposed_filtered_zip.reset_index()

# Rename the first column to "Zip" instead of "Date"
transposed_filtered_zip = transposed_filtered_zip.rename(columns={'index': 'Zip'})

# Set the dates as column names
transposed_filtered_zip.columns = transposed_filtered_zip.iloc[0]
transposed_filtered_zip = transposed_filtered_zip.drop(transposed_filtered_zip.index[0])

transposed_filtered_zip = transposed_filtered_zip.rename(columns={'Date': 'Zip'})

# Add the "Coastal/Inland" and "Pair" columns
transposed_filtered_zip['Coastal/Inland'] = transposed_filtered_zip['Zip'].apply(lambda x: 1 if x in Coastal else 0)
transposed_filtered_zip['Pair'] = 0
pair_index = 1

for i in range(0, len(Combined), 2):
    inland_zip = Combined[i]
    coastal_zip = Combined[i+1]
    transposed_filtered_zip.loc[transposed_filtered_zip['Zip'] == inland_zip, 'Pair'] = pair_index
    transposed_filtered_zip.loc[transposed_filtered_zip['Zip'] == coastal_zip, 'Pair'] = pair_index
    pair_index += 1

# Sort the DataFrame by the "Pair" column
transposed_filtered_zip = transposed_filtered_zip.sort_values(by='Pair')

In [3]:
# Save the transposed DataFrame to a new CSV file
transposed_filtered_zip.to_csv('inputs/transposed_filtered_zip.csv', index=False)
print(transposed_filtered_zip.head())

0                           Zip 2010-01 2010-02 2010-03 2010-04 2010-05  \
43  SmoothedGSML_GIA_sigremoved   14.64   15.98    15.0   14.99   16.93   
41               StdDevGMSL_GIA   84.58   87.21   85.81   84.26    84.8   
40                     GMSL_GIA   12.43   17.27   11.56   11.65   14.47   
39           SmoothedGSML_noGIA    9.54   10.16    8.53    7.51    8.67   
38             StdDevGMSL_noGIA   84.67    87.3   86.05    84.6   85.01   

0  2010-06 2010-07 2010-08 2010-09  ... 2017-02 2017-03 2017-04 2017-05  \
43   17.66   15.38    12.1   10.46  ...   41.66    43.1    43.1   42.83   
41   84.35    82.0   86.31   90.33  ...   82.05   87.69   85.58   82.17   
40   15.51   14.92   15.56   16.53  ...   40.48    41.7    40.3   35.98   
39    9.95   10.02   10.05    10.9  ...   34.12    34.9   33.88   32.87   
38   84.38   81.92   85.96   89.98  ...    82.1   87.94   85.64   82.17   

0  2017-06 2017-07 2017-08 2017-09 Coastal/Inland Pair  
43   43.57   44.51   45.51   45.28       

In [125]:
#Melt dataset - Rearrange data
transposed_filtered_zip = transposed_filtered_zip.iloc[8:]
melted_df = transposed_filtered_zip.melt(id_vars=['Zip', 'Coastal/Inland', 'Pair'],
                    var_name='Date',
                    value_name='Price')

#Prep sea level data for merge
filtered_data = filtered_data.loc['2010-01':'2017-09'].reset_index()

In [126]:
#merge:
zip_sea = pd.merge(melted_df,filtered_data.rename(columns={'Year': 'Date'}),
         on='Date',   how='outer', validate='many_to_one')

zip_sea.to_csv('inputs/zip_sea.csv', index=False)
zip_sea

,Zip,Coastal/Inland,Pair,Date,Price,TotalWeightedObservations,GMSL_noGIA,StdDevGMSL_noGIA,SmoothedGSML_noGIA,GMSL_GIA,StdDevGMSL_GIA,SmoothedGSML_GIA,SmoothedGSML_GIA_sigremoved
0,94014.0,0,1,2010-01,NaN,336505.31,8.3,84.67,9.54,12.43,84.58,13.67,14.64
1,94015.0,1,1,2010-01,NaN,336505.31,8.3,84.67,9.54,12.43,84.58,13.67,14.64
2,93940.0,0,2,2010-01,NaN,336505.31,8.3,84.67,9.54,12.43,84.58,13.67,14.64
3,93950.0,1,2,2010-01,749900.0,336505.31,8.3,84.67,9.54,12.43,84.58,13.67,14.64
4,93109.0,1,3,2010-01,NaN,336505.31,8.3,84.67,9.54,12.43,84.58,13.67,14.64
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3250,11209.0,1,19,2017-09,748000.0,333942.19,45.5,86.53,44.07,51.54,87.08,50.10,45.28
3251,10023.0,0,20,2017-09,2395000.0,333942.19,45.5,86.53,44.07,51.54,87.08,50.10,45.28
3252,10069.0,1,20,2017-09,2195000.0,333942.19,45.5,86.53,44.07,51.54,87.08,50.10,45.28
3253,10003.0,0,21,2017-09,2495000.0,333942.19,45.5,86.53,44.07,51.54,87.08,50.10,45.28
